# Initialization

In [1]:
import polars as pl
import polars.selectors as cs

from pathlib import Path
import os
import re

In [2]:
files_clean = r'C:\Users\mpola\asthma analysis\files\cleaned files'
files_clean_path = Path(files_clean)

csv_files = list(files_clean_path.glob('*.csv'))

csv_names_list = [file.stem.replace('.csv', '') for file in csv_files]

In [3]:
df_list = {}

for i, entry in enumerate(csv_files):
    df_list[csv_names_list[i]] = pl.read_csv(entry, infer_schema_length=2000, ignore_errors=True)

In [4]:
csv_names_list

['5yr_demographic',
 '5yr_private_insurance',
 '5yr_public_insurance',
 'asthma',
 'counties',
 'county_additional_measures',
 'county_health_measures',
 'lat_long',
 'md_per_state',
 'pollution',
 'resp_therapist_per_state',
 'svi_2018']

# Analysis

In [5]:
# Our datasets have a LOT of data, some of which is of no use to us due to their
# colinearity with already existing data. A cleanup makes things faster and easier to interpret.

def batch_drop_redundant_features(dfs_dict, threshold=0.95, ignore_cols=None, target_keys=None):
    """
    Compares data within datasets and drops colinear columns.
    
    Parameters:
    - dfs_dict: Dictionary of Polars DataFrames.
    - threshold: Correlation threshold (0.0 to 1.0).
    - ignore_cols: List of column names to exclude from the check (e.g., IDs).
    - target_keys: List of dictionary keys (strings). If provided, only these 
                   DataFrames will be processed. Others will be kept as-is.
    """
    if ignore_cols is None:
        ignore_cols = []
    
    cleaned_dict = {}

    print(f'--- Starting Redundancy Check (Threshold: {threshold}) ---')

    for name, df in dfs_dict.items():
        # If a target list is provided and the current key isn't in it, 
        # we keep the dataframe as is and skip the logic.
        if target_keys is not None and name not in target_keys:
            # Optional: Print that we are skipping it
            # print(f'[{name}] Skipping (not in target list).') 
            cleaned_dict[name] = df
            continue

        # Identifying numeric columns to check, excluding IDs
        numeric_cols = [
            c for c in df.select(pl.col(pl.NUMERIC_DTYPES)).columns 
            if c not in ignore_cols
        ]
        
        # If dataset has no numeric columns to check, we keep it as is
        if not numeric_cols:
            cleaned_dict[name] = df
            continue

        # Computing correlation matrix for these columns only
        corr_matrix = df.select(numeric_cols).corr()
        cols = corr_matrix.columns
        
        to_drop = set()

        # Iterating the upper triangle of the matrix
        for i in range(len(cols)):
            col_a = cols[i]
            
            if col_a in to_drop:
                continue

            for j in range(i + 1, len(cols)):
                col_b = cols[j]
                
                if col_b in to_drop:
                    continue
                
                val = corr_matrix[i, j]
                
                if val is not None and abs(val) >= threshold:
                    to_drop.add(col_b)
        
        # Dropping the columns and save to new dict
        if to_drop:
            print(f'[{name}] Dropping {len(to_drop)} columns...')
            for i in to_drop:
                print(i)
            cleaned_dict[name] = df.drop(list(to_drop))
        else:
            print(f'[{name}] No redundant columns found.')
            cleaned_dict[name] = df
        print('')

    return cleaned_dict

In [6]:
id_columns = ['County (State)', 'County', 'State', 'Latitude', 'Longitude']

# Running the cleaning
df_list_dropped = {}
df_list_dropped = batch_drop_redundant_features(
    df_list, 
    threshold=0.98, 
    ignore_cols=id_columns
)

--- Starting Redundancy Check (Threshold: 0.98) ---


C:\Users\mpola\AppData\Local\Temp\ipykernel_7040\3312401638.py:33: DeprecationWarning: `NUMERIC_DTYPES` was deprecated in version 1.0.0. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  c for c in df.select(pl.col(pl.NUMERIC_DTYPES)).columns


[5yr_demographic] Dropping 17 columns...
50 to 54 years
5 to 9 years
70 to 74 years
15 to 19 years
10 to 14 years
25 to 29 years
85 years and over
20 to 24 years
30 to 34 years
40 to 44 years
Under 5 years
55 to 59 years
60 to 64 years
35 to 39 years
65 to 69 years
45 to 49 years
80 to 84 years

[5yr_private_insurance] Dropping 1 columns...
COVERAGE ALONE OR IN COMBINATION => Tricare/military  insurance alone or in combination => 19 to 64 years

[5yr_public_insurance] Dropping 1 columns...
COVERAGE ALONE => Public insurance alone => Medicaid/means tested coverage alone

[asthma] No redundant columns found.

[county_additional_measures] No redundant columns found.

[county_health_measures] Dropping 1 columns...
Some college => Population

[md_per_state] No redundant columns found.

[pollution] No redundant columns found.

[resp_therapist_per_state] No redundant columns found.



C:\Users\mpola\AppData\Local\Temp\ipykernel_7040\3312401638.py:33: DeprecationWarning: `NUMERIC_DTYPES` was deprecated in version 1.0.0. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  c for c in df.select(pl.col(pl.NUMERIC_DTYPES)).columns


[svi_2018] Dropping 27 columns...
(General) Sum of series themes [SPL_THEMES]
(Housing Type/Transportation) Sum of flags for the four themes [F_TOTAL]
(Socioeconomic) Flag - the percentage of civilian unemployed is in the 90th percentile (1 = yes, 0 = no) 2018 DESCRIPTION [F_UNEMP]
(Socioeconomic) Percentile Percentage of persons below poverty estimate [EPL_POV]
(Socioeconomic) Flag - per capita income is in the 90th percentile (1 = yes, 0 = no) [F_PCI]
(Socioeconomic) Civilian (age 16+) unemployed estimate, 2014-2018 ACS [E_UNEMP]
(Household Composition/Disability) Civilian noninstitutiona lized population with a disability estimate, 2014-2018 ACS [E_DISABL]
(Socioeconomic) Percentile ranking for Socioeconomic theme summary [RPL_THEME1]
(Minority Status/Language) Minority (all persons except white, non- Hispanic) estimate, 2014-2018 ACS [E_MINRTY]
(Housing Type/Transportation) Percentile ranking for Housing Type/ Transportation theme [RPL_THEME4]
(Socioeconomic) Unemploymen t Rate est

## Merging

In [7]:
# Merging all dataframes into one and dropping string columns beside state and county

# While we will join by the County (State) column, the respiratory therapist
# and MD datasets are still recorded on a per-state basis and will require an exception
# to be made
state_level_keys = ['resp_therapist_per_state', 'md_per_state']


# We will use the counties table as the base table since it covers all counties present in other dataframes

df_base = df_list_dropped.pop('counties').lazy()


for name, df_next in df_list_dropped.items():
    df_lazy = df_next.lazy()
    
    if name in state_level_keys:
        # State-Level Join
        # Joining on State Code only. 
        # This broadcasts state info to every county in that state.
        df_base = df_base.join(
            df_lazy,
            on='State',
            how='left',
            suffix=f'_{name}'
        )
    else:
        # County-Level Join
        # Joining on new composite key 'County (State)'
        # This prevents 'Orange (California)' from matching with 'Orange (Florida)'
        df_clean = df_lazy.select(pl.exclude(['State', 'County']))

        df_base = df_base.join(
            df_clean,
            on='County (State)', 
            how='left',
            suffix=f'_{name}'
        )

# Finalizing
df_base = df_base.collect()
df_base


County (State),County,State,Geography,Geographic Area Name,Total population,75 to 79 years,Median age (years),Sex ratio (males per 100 females),Age dependency ratio,Old-age dependency ratio,Child dependency ratio,Under 18 %age Estimate,Geography_5yr_private_insurance,Geographic Area Name_5yr_private_insurance,COVERAGE ALONE OR IN COMBINATION => Employer-based insurance alone or in combination,COVERAGE ALONE OR IN COMBINATION => Employer-based insurance alone or in combination => Under 19,COVERAGE ALONE OR IN COMBINATION => Employer-based insurance alone or in combination => 19 to 64 years,COVERAGE ALONE OR IN COMBINATION => Employer-based insurance alone or in combination => 65 years and over,COVERAGE ALONE OR IN COMBINATION => Direct-purchase insurance alone or in combination,COVERAGE ALONE OR IN COMBINATION => Direct-purchase insurance alone or in combination => Under 19,COVERAGE ALONE OR IN COMBINATION => Direct-purchase insurance alone or in combination => 19 to 64 years,COVERAGE ALONE OR IN COMBINATION => Direct-purchase insurance alone or in combination => 65 years and over,COVERAGE ALONE OR IN COMBINATION => Tricare/military insurance alone or in combination,COVERAGE ALONE OR IN COMBINATION => Tricare/military insurance alone or in combination => Under 19,COVERAGE ALONE OR IN COMBINATION => Tricare/military insurance alone or in combination => 65 years and over,PRIVATE INSURANCE ALONE OR IN COMBINATION => Below 138 percent of the poverty threshold,PRIVATE INSURANCE ALONE OR IN COMBINATION => At or above 138 percent of the poverty threshold,"PRIVATE INSURANCE ALONE OR IN COMBINATION => Worked full-time, year-round (19-64 years)",PRIVATE INSURANCE ALONE OR IN COMBINATION => Under 6,PRIVATE INSURANCE ALONE OR IN COMBINATION => 6 to 18 years,PRIVATE INSURANCE ALONE OR IN COMBINATION => 19 to 25 years,PRIVATE INSURANCE ALONE OR IN COMBINATION => 26 to 34 years,PRIVATE INSURANCE ALONE OR IN COMBINATION => 35 to 44 years,PRIVATE INSURANCE ALONE OR IN COMBINATION => 45 to 54 years,PRIVATE INSURANCE ALONE OR IN COMBINATION => 55 to 64 years,PRIVATE INSURANCE ALONE OR IN COMBINATION => 65 to 74 years,…,"(Housing Type/Transportation) Percentage of persons in group quarters estimate, 2014-2018 ACS 2018 DESCRIPTION [EP_GROUPQ]","(Housing Type/Transportation) Percentage of persons in group quarters estimate MOE, 2014-2018 ACS [MP_GROUPQ]",(Socioeconomic) Percentile Percentage of persons with no high school diploma (age 25+) estimate [EPL_NOHSDP],(Household Composition/Disability) Percentile percentage of persons aged 65 and older estimate [EPL_AGE65],(Household Composition/Disability) Percentile percentage of persons aged 17 and younger estimate [EPL_AGE17],(Household Composition/Disability) Percentile percentage of civilian noninstitutiona lized population with a disability estimate 2018 DESCRIPTION [EPL_DISABL],(Household Composition/Disability) Percentile percentage of single parent households with children under 18 estimate [EPL_SNGPNT],(Household Composition/Disability) Sum of series for Household Composition theme [SPL_THEME2],(Household Composition/Disability) Percentile ranking for Household Composition theme summary [RPL_THEME2],"(Minority Status/Language) Percentile percentage minority (all persons except white, non- Hispanic) estimate [EPL_MINRTY]","(Minority Status/Language) Percentile percentage of persons (age 5+) who speak English """"less than well"""" estimate [EPL_LIMENG]",(Minority Status/Language) Sum of series for Minority Status/Languag e theme 2018 DESCRIPTION [SPL_THEME3],(Housing Type/Transportation) Percentile percentage housing in structures with 10 or more units estimate [EPL_MUNIT],(Housing Type/Transportation) Percentile percentage mobile homes estimate [EPL_MOBILE],(Housing Type/Transportation) Percentile percentage households with more people than rooms estimate [EPL_CROWD],(Housing Type/Transportation) Percentile percentage households with no vehicle available estimate [EPL_NOVEH],(Housing Type/Transp

## Additional Data Cleaning

In [8]:
asthma_columns = ['Age-adjusted ER Visit Rate for Asthma per 10,000 People',
                  'Age-adjusted Hospitalization Rate for Asthma per 10,000 People']

In [9]:
# Dropping all string columns besides the county and state information

# Defining the string columns we strictly want to KEEP
columns_to_keep = ['County (State)', 'County', 'State']

# Performing the selection
# Logic: Select (Anything that is NOT a String) OR (Anything inside the keep list)
df_final = df_base.select(
    ~cs.string() | cs.by_name(columns_to_keep)
)


# Dropping the columns with too many missing entries (more than 30%)

total_rows = df_final.height
threshold = 0.3

# Getting null counts for all columns at once
null_stats = df_final.null_count().row(0, named=True)

cols_to_keep = [
    col_name for col_name, null_count in null_stats.items()
    if (null_count / total_rows) < threshold
]


# Ensuring we keep the asthma-related columns
final_cols = list(set(cols_to_keep + asthma_columns))

df_final = df_final.select(final_cols)



In [10]:
df_final

(Household Composition/Disability) Percentile percentage of single parent households with children under 18 estimate [EPL_SNGPNT],PUBLIC INSURANCE ALONE OR IN COMBINATION => 45 to 54 years,"(Household Composition/Disability) Percentage of persons aged 65 and older estimate, 2014-2018 ACS [EP_AGE65]",(Socioeconomic) Percentile Percentage of persons with no high school diploma (age 25+) estimate [EPL_NOHSDP],Value_Pollutant: Formaldehyde,Sexually transmitted infections => Chlamydia Rate,COVERAGE ALONE OR IN COMBINATION => Tricare/military insurance alone or in combination,Access to exercise opportunities => % With Access,Longitude,"(Housing Type/Transportation) Flag - the percentage of households with no vehicles is in the 90th percentile (1 = yes, 0 = no) [F_NOVEH]",Demographics => % < 18,Excessive drinking => % Excessive Drinking,"(Socioeconomic) Flag - the percentage of persons with no high school diploma is in the 90th percentile (1 = yes, 0 = no) [F_NOHSDP]",(Socioeconomic) Percentage of persons with no high school diploma (age 25+) estimate [EP_NOHSDP],PUBLIC INSURANCE ALONE OR IN COMBINATION => 75 years and over,Diabetes prevalence => % Diabetic,Severe housing problems => Inadequate Facilities,PUBLIC INSURANCE ALONE OR IN COMBINATION => Below 138 percent of the poverty threshold,COVERAGE ALONE OR IN COMBINATION => Tricare/military insurance alone or in combination => 65 years and over,"PUBLIC INSURANCE ALONE OR IN COMBINATION => Worked full-time, year-round (19-64 years)",COVERAGE ALONE OR IN COMBINATION => Direct-purchase insurance alone or in combination => Under 19,"(Household Composition/Disability) Flag - the percentage of single parent households is in the 90th percentile (1 = yes, 0 = no) [F_SNGPNT]","Population, All (State Level File)",Adult smoking => % Smokers,Children in poverty => % Children in Poverty (Hispanic),"(General) Adjunct variable - Uninsured in the total civilian noninstitutiona lized population estimate MOE, 2014-2018 ACS [M_UNINSUR]",Demographics => % Female,Children in poverty => % Children in Poverty (White),PUBLIC INSURANCE ALONE OR IN COMBINATION => 65 to 74 years,Severe housing problems => % Severe Housing Problems,COVERAGE ALONE => Private insurance alone => Employer-based insurance alone,"M.D., Active M.D. (County Level File)","Value_Pollutant: 1,3-butadiene","(Household Composition/Disability) Civilian noninstitutiona lized population with a disability estimate MOE, 2014-2018 ACS [M_DISABL]",(Housing Type/Transportation) Percentage of households with no vehicle available estimate [EP_NOVEH],Limited access to healthy foods => # Limited Access,"(Housing Type/Transportation) Households with no vehicle available estimate, 2014-2018 ACS [E_NOVEH]",…,"(Housing Type/Transportation) Flag - the percentage of households in multi-unit housing is in the 90th percentile (1 = yes, 0 = no) 2018 DESCRIPTION [F_MUNIT]",Residential segregation - non-white/white => Segregation Index,Poor physical health days => Physically Unhealthy Days,(Housing Type/Transportation) Percentage of housing in structures with 10 or more units estimate 2018 DESCRIPTION [EP_MUNIT],COVERAGE ALONE OR IN COMBINATION => VA care coverage alone or in combination => 19 to 64 years,Value_Pollutant: Ethylene oxide,COVERAGE ALONE OR IN COMBINATION => Employer-based insurance alone or in combination => Under 19,Uninsured => % Uninsured,Violent crime => Annual Average Violent Crimes,Latitude,"(General) Housing units estimate MOE, 2014-2018 ACS [M_HU]",Old-age dependency ratio,Income inequality => 80th Percentile Income,Demographics => % Native Hawaiian/Other Pacific Islander,"(Household Composition/Disability) Flag - the percentage of persons aged 65 and older is in the 90th percentile (1 = yes, 0 = no) [F_AGE65]",COVERAGE ALONE OR IN COMBINATION => Medicare coverage alone or in combination => 19 to 64 years,(Housing Type/Transportation) Percentile percentage mobile homes estimate [EPL_MOBILE],Demographics => # American Indian/Alaskan Native

## Imputing 

In [11]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.preprocessing import StandardScaler

# Since our datasets don't fully cover every county, we have to either drop problematic features or fill in the gaps
# ourselves. We've already dropped the features that were missing from at least 30% of the counties in the previous step,
# and now will impute the rest using the existing data. We will ultimately work on roughly half of our counties, since
# our target features are asthma-related ER visits and hospitalizations (which we won't impute in any way), so even the counties
# that have no asthma-related data are of importance to us as they potentially form a baseline for the imputation of missing
# features for the counties that do have asthma data

def smart_spatial_imputation(df, 
                             hard_constraint_cols, 
                             target_cols=None,
                             state_column='State', 
                             index_column='County (State)'):
    
    # Handling the mutable default argument
    if target_cols is None:
        target_cols = []

    # Polars conversion to Pandas (easier to work with using sklearn and numpy)
    if hasattr(df, 'to_pandas'):
        df = df.to_pandas()
    
    if index_column in df.columns:
        df = df.set_index(index_column)

    ################# Separation step #################
    # Isolating the target columns so they are untouched by scaling/imputation
    # We use 'reindex' to handle cases where a target might not exist or be misnamed safely
    df_targets = df[target_cols].copy()
    
    # Scaling
    # Selecting numeric features, EXCLUDING the targets
    numeric_df = df.select_dtypes(include=['number'])
    features_to_impute = [c for c in numeric_df.columns if c not in target_cols]
    numeric_df = numeric_df[features_to_impute]

    scaler = StandardScaler()
    scaled_array = scaler.fit_transform(numeric_df)
    
    working_df = pd.DataFrame(
        scaled_array, 
        columns=numeric_df.columns, 
        index=numeric_df.index
    )
    
    # Re-attaching state for Phase 1 grouping
    working_df[state_column] = df[state_column]

    ################# Hard Constraints (State-Bound KNN) #################
    # For features more likely to be impacted by state-level legislations, we use an isolated
    # K-nearest neighbors algorithm that ignores counties not belonging to the same state
    # How we determine these features is explained in the below function.
    knn_state = KNNImputer(n_neighbors=5, weights='distance')
    numeric_cols = numeric_df.columns.tolist()

    if hard_constraint_cols:
        for state_name, state_subset in working_df.groupby(state_column):
            
            current_indices = state_subset.index
            state_chunk_numeric = state_subset[numeric_cols]
            
            # Skipping if the whole state is empty
            if state_chunk_numeric.isnull().all().all():
                continue

            # Identifying columns that have at least SOME data in this state.
            valid_cols = state_chunk_numeric.columns[state_chunk_numeric.notna().any()].tolist()
            
            # If there are no valid columns, we skip
            if not valid_cols:
                continue

            # Filtering the chunk to ONLY valid columns
            data_to_impute = state_chunk_numeric[valid_cols]
            
            # Imputing
            imputed_array = knn_state.fit_transform(data_to_impute)
            
            # Creating DataFrame using ONLY the valid columns
            imputed_df = pd.DataFrame(
                imputed_array, 
                columns=valid_cols, 
                index=current_indices
            )
            
            # Updating only the hard constraint columns that existed in this state
            for col in hard_constraint_cols:
                # Ensuring we don't try to update a target column accidentally (double check)
                if col in valid_cols and col not in target_cols: 
                    working_df.loc[current_indices, col] = imputed_df[col]

    ################# Spatial MICE (Global) #################
    # For all other features, we use multiple imputation by chained equations.
    # Since we have latitude/longitude data from the USCB Gazetteer file and the state-level
    # dummies from the previous function, this imputing also accounts for spatial positioning and,
    # to some extent, state-level differences.
    # Dropping state column so everything is numeric
    global_input = working_df.drop(columns=[state_column])
    
    # Running MICE
    mice = IterativeImputer(max_iter=10, random_state=0)
    global_imputed_array = mice.fit_transform(global_input)

    ################# Reconstruction #################
    final_numeric_array = scaler.inverse_transform(global_imputed_array)
    
    df_numeric_final = pd.DataFrame(
        final_numeric_array,
        columns=numeric_df.columns,
        index=numeric_df.index
    )
    
    # Creating the state-level dummies
    state_dummies = pd.get_dummies(df[state_column], prefix='State', drop_first=True)
    
    # Concatenating the imputed features, target features and state-level dummies together
    return pd.concat([df_numeric_final, df_targets, state_dummies], axis=1)

In [12]:
def get_hard_constraint_cols(df, keywords):
    """
    Returns a list of column names that contain any of the keywords (case-insensitive).
    """
    # Getting all column names
    all_cols = df.columns
    
    # Finding matches
    # We use .lower() to ensure 'Medicaid' matches 'medicaid'
    matches = [
        col for col in all_cols 
        if any(keyword.lower() in col.lower() for keyword in keywords)
    ]
    
    return list(set(matches)) # Remove duplicates just in case

# Defining the words that imply a state-level policy boundary
policy_keywords = [
    'medicaid', 'insurance', 'enrollment', 'medicare'               # Health Policy-related
    'tax', 'spending', 'funding',                                   # Fiscal Policy-related
    'graduation', 'proficiency', 'score', 'college', 'education'    # Education, since people are more likely to go to universities in-state to avoid high tuition
    'crime', 'arrest'                                               # Legal/Reporting Standards
]

# Generating the list automatically
hard_cutoff_list = get_hard_constraint_cols(df_final, policy_keywords)

print(f'Found {len(hard_cutoff_list)} columns to apply hard state constraints:')
for i in hard_cutoff_list:
    print(i)

Found 50 columns to apply hard state constraints:
COVERAGE ALONE OR IN COMBINATION => Tricare/military  insurance alone or in combination => Under 19
COVERAGE ALONE => Public insurance alone
High school graduation => Cohort Size
COVERAGE ALONE OR IN COMBINATION => Direct-purchase insurance alone or in combination
PRIVATE INSURANCE ALONE OR IN COMBINATION => 35 to 44 years
PUBLIC INSURANCE ALONE OR IN COMBINATION => 45 to 54 years
PRIVATE INSURANCE ALONE OR IN COMBINATION => 65 to 74 years
PRIVATE INSURANCE ALONE OR IN COMBINATION => 6 to 18 years
PRIVATE INSURANCE ALONE OR IN COMBINATION => 55 to 64 years
COVERAGE ALONE => Private insurance alone
COVERAGE ALONE => Private insurance alone => Direct-purchase insurance alone
COVERAGE ALONE OR IN COMBINATION => Employer-based insurance alone or in combination
COVERAGE ALONE OR IN COMBINATION => Tricare/military  insurance alone or in combination
PUBLIC INSURANCE ALONE OR IN COMBINATION => At or above 138 percent of the poverty threshold
PR

In [13]:
df_imputed = smart_spatial_imputation(df_final, 
                                      hard_constraint_cols = hard_cutoff_list,
                                      target_cols = asthma_columns)

c:\Users\mpola\asthma analysis\.conda\Lib\site-packages\sklearn\impute\_iterative.py:867: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


# Collinearity Analysis

In [14]:
# Calculating basic correlation/collinearity using Pandas

# df_imputed = df_imputed.to_pandas()

scaler = StandardScaler().set_output(transform='pandas')

df_scaled = scaler.fit_transform(df_imputed)

corr_matrix = df_scaled.corr(method='pearson')



In [15]:
# Creating a mask to hide the lower triangle and diagonal (duplicates & self-correlation)
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)

# Applying the mask and stack (flatten) the result
# 'stack' converts the grid into a Series multi-indexed by (Row, Col)
strong_pairs = (
    corr_matrix.where(mask)
    .stack()
    .reset_index()
)

# Renaming columns and sorting by strength (absolute value)
strong_pairs.columns = ['Variable_1', 'Variable_2', 'Correlation']
strong_pairs['Abs_Correlation'] = strong_pairs['Correlation'].abs()


significant_findings = strong_pairs.sort_values('Abs_Correlation', ascending=False)

# This result has a lot of redundant matches since some of our databases had duplicate entries (multiple tracked insurance status, for example)
# but won't form the basis of our in-depth analysis anyway
print(significant_findings.head(20))

                                              Variable_1  \
34038                         Demographics => Population   
34123                         Demographics => Population   
43638                                   Total population   
7064                  Population, All (State Level File)   
11593  (Minority Status/Language) Persons (age 5+) wh...   
41944                           Uninsured => # Uninsured   
47404  (General) Adjunct variable - Uninsured in the ...   
41921                           Uninsured => # Uninsured   
38028                        Unemployment => Labor Force   
38113                        Unemployment => Labor Force   
34004                         Demographics => Population   
18259               Demographics => % Non-Hispanic White   
43598                                   Total population   
34083                         Demographics => Population   
49173  Children in single-parent households => # Hous...   
48329  Alcohol-impaired driving deaths =

In [16]:
# Defining the feature groups in a dictionary
# Since Python iterates through the dictionary until it finds a match, we want the
# more crucial and defining categories higher on the list.
# These groups exist to make parsing the output easier, and don't have a direct impact on the statistical model itself

# Some terms like 'diabet' and 'insur' are deliberately cut off to catch the variations of the same word, 
# like 'diabet-ic' and 'diabet-es' or 'insur-ance' and 'insur-ed'

term_groups = {
    'Asthma':         ['asthma'],
    'Pollution':      ['pollutant', 'pollution'],
    'Care Provider':  ['respiratory therapist', 'm.d.', 'physician', 'hospital'],
    'Insurance':      ['insur', 'coverage', 'medicaid', 'private', 'medicare'],
    'Education':      ['education', 'college', 'university', 'school'],

    'Other Health':   ['physical', 'smok', 'disability', 'disabl', 'medical', 'mental', 
                       'activity', 'hiv', 'diabet', 'food', 'nutrition', 'injury', 
                       'mortality', 'obese', 'life expectancy', 'birthweight', 'alcohol'],

    'Socioeconomic':  ['income', 'poverty', 'unemployment', 'education', 'socioeconomic', 
                       'minority', 'demographic', 'crime', 'social', 
                       'alone', 'single', 'population'],
                       
    'Housing':        ['rent', 'value', 'owner', 'vacant', 'crowd', 'hous'],
}


In [17]:
def get_group(variable_name, groups_dict):
    """
    Returns the name of the group a variable belongs to, or None.
    """
    var_lower = variable_name.lower()
    for group_name, terms in groups_dict.items():
        if any(term in var_lower for term in terms):
            return group_name
    return 'Unspecified'

def organize_interactions(row, groups_dict):
    v1 = row['Variable_1']
    v2 = row['Variable_2']
    
    g1 = get_group(v1, groups_dict)
    g2 = get_group(v2, groups_dict)
    
    # CASE 1: Different Groups (The Standard Case)
    # We sort them alphabetically so 'Asthma vs Pollution' and 'Pollution vs Asthma'
    # both become 'Asthma vs Pollution'.
    if g1 != g2:
        if g1 < g2:
            return pd.Series([f'{g1} vs {g2}', v1, v2])
        else:
            # Swap variables so the first column matches the first group name
            return pd.Series([f'{g2} vs {g1}', v2, v1])

    # CASE 2: Same Group (e.g. Unspecified vs Unspecified)
    # We label them as Internal so we don't miss data.
    else:
        return pd.Series([f'{g1} (Internal)', v1])


# Applying the function. Note that we assign it to THREE columns at once.
significant_findings[['Interaction', 'Variable_1', 'Variable_2']] = significant_findings.apply(
    organize_interactions, 
    axis=1, 
    groups_dict=term_groups
)

cross_group_correlations = significant_findings
cross_group_correlations['Interaction'] = cross_group_correlations['Interaction'].fillna('Unspecified')

# Viewing specific interactions (e.g., Pollution impacting Health)
print(cross_group_correlations.sort_values('Interaction').head(20))

                                              Variable_1  \
52854  Age-adjusted ER Visit Rate for Asthma per 10,0...   
47791  Age-adjusted Hospitalization Rate for Asthma p...   
17916  Age-adjusted ER Visit Rate for Asthma per 10,0...   
45586  Age-adjusted Hospitalization Rate for Asthma p...   
9978   Age-adjusted ER Visit Rate for Asthma per 10,0...   
29701  Age-adjusted Hospitalization Rate for Asthma p...   
47790  Age-adjusted ER Visit Rate for Asthma per 10,0...   
52691  Age-adjusted ER Visit Rate for Asthma per 10,0...   
29700  Age-adjusted ER Visit Rate for Asthma per 10,0...   
52692  Age-adjusted Hospitalization Rate for Asthma p...   
45585  Age-adjusted ER Visit Rate for Asthma per 10,0...   
17917  Age-adjusted Hospitalization Rate for Asthma p...   
9979   Age-adjusted Hospitalization Rate for Asthma p...   
27435  Age-adjusted ER Visit Rate for Asthma per 10,0...   
4145   Age-adjusted ER Visit Rate for Asthma per 10,0...   
14891  Age-adjusted ER Visit Rate for As

In [18]:
# Printing out all the interactions for manually looking at the results.
# We will also save the correlations in a table in the end, so this step only
# exists as a troubleshooting option

var1 = 'Variable_1'
var2 = 'Variable_2'
coef = 'Correlation'
lower_threshold = 0.3
upper_threshold = 0.9
count = 20

# Getting unique interactions (sorted) so the output groups are in order

unique_interactions = cross_group_correlations['Interaction'].unique()

unique_interactions.sort()

for i in unique_interactions:
    print(f'#=== Now looking at {i} ===#')
    
    # Filtering to get specific group and applying threshold
    filtered_df = cross_group_correlations[
        (cross_group_correlations['Interaction'] == i) & 
        (cross_group_correlations['Abs_Correlation'] > lower_threshold) &
        (cross_group_correlations['Abs_Correlation'] < upper_threshold)
    ]
    
    # Sorting by strength (descending) and taking the top 'count'
    top_results = filtered_df.sort_values(by='Abs_Correlation', ascending=False).head(count)

    # Printing the results
    for row in top_results.itertuples():
        print(f'**Feature A:** {getattr(row, var1)}  ')
        print(f'**Feature B:** {getattr(row, var2)}  ')
        print(f'**Correlation coefficient of A and B:** {getattr(row, coef):.3f}  ')
        print('  ')
    print('\n\n')

#=== Now looking at Asthma (Internal) ===#
**Feature A:** Age-adjusted ER Visit Rate for Asthma per 10,000 People  
**Feature B:** nan  
**Correlation coefficient of A and B:** 0.652  
  



#=== Now looking at Asthma vs Care Provider ===#



#=== Now looking at Asthma vs Education ===#
**Feature A:** Age-adjusted Hospitalization Rate for Asthma per 10,000 People  
**Feature B:** (Socioeconomic) Persons (age 25+) with no high school diploma estimate MOE, 2014-2018 ACS [M_NOHSDP]  
**Correlation coefficient of A and B:** 0.397  
  
**Feature A:** Age-adjusted ER Visit Rate for Asthma per 10,000 People  
**Feature B:** (Socioeconomic) Persons (age 25+) with no high school diploma estimate MOE, 2014-2018 ACS [M_NOHSDP]  
**Correlation coefficient of A and B:** 0.368  
  
**Feature A:** Age-adjusted ER Visit Rate for Asthma per 10,000 People  
**Feature B:** High school graduation => Graduation Rate  
**Correlation coefficient of A and B:** -0.315  
  



#=== Now looking at Asthma vs Hous

# LASSO, OLS and VIF
This is the step in which we establish the statistical models used. We use LASSO to select relevant features, calculate the ordinary least squares, variance inflation factors and p-values to check whether a feature reliably explains the rise in ER visits and hospitalization rates by itself.

In [19]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.pipeline import make_pipeline

# Container for our improved analysis
significance_results = {}

# X = df_imputed.drop(columns=asthma_columns)

for target in asthma_columns:
    print(f'\nAnalyzing Significance for: {target}')


    df_iter = df_imputed.dropna(subset=[target]).copy()
    print(f'Counties available for this model: {len(df_iter)}')


    y = df_iter[target].values
    X = df_iter.drop(columns=asthma_columns)

    ################# Standardizing and preparign the data #################

    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)
    
    # Running LASSO for Feature Selection
    # We use LassoCV to automatically find the best alpha (penalty)
    lasso = LassoCV(cv=5, random_state=2211, max_iter=50000).fit(X_scaled, y)
    
    # Getting the features where coefficient is NOT zero
    selected_mask = lasso.coef_ != 0
    selected_features = X.columns[selected_mask].tolist()
    
    if len(selected_features) == 0:
        print('LASSO selected 0 features. Cannot run OLS.')
        continue
        
    print(f'LASSO selected {len(selected_features)} features. Running OLS for significance...')
    
    ################# Post-LASSO OLS (Unpenalized Regression on selected features) #################
    # We slice X to only the selected features
    X_subset = X_scaled[selected_features].copy()
    X_subset = sm.add_constant(X_subset) # Add intercept for OLS
    
    ols_model = sm.OLS(y, X_subset).fit()
    
    # We use adjusted R^2 since the count of features included in each model is not consistent
    print(f'Adjusted R^2: {ols_model.rsquared_adj:.4f}')

    ################# Calculating Collinearity (VIF) #################
    # VIF > 5 indicates high collinearity
    vif_data = pd.DataFrame()
    vif_data['Feature'] = X_subset.columns
    vif_data['VIF'] = [variance_inflation_factor(X_subset.values, i) 
                       for i in range(X_subset.shape[1])]
    
    ################# Compiling results #################
    # Extracting p-values and coefficients from OLS
    summary_df = pd.DataFrame({
        'Feature': selected_features,
        'LASSO_Coef': lasso.coef_[selected_mask], # Original penalized coef
        'OLS_Coef': ols_model.params[1:].values,  # Unpenalized coef
        'P_Value': ols_model.pvalues[1:].values,  # Significance
        'Conf_Int_Low': ols_model.conf_int()[1:][0].values,
        'Conf_Int_High': ols_model.conf_int()[1:][1].values
    })
    
    # Merging VIF data (excluding const)
    summary_df = summary_df.merge(vif_data, on='Feature', how='left')

    # Filtering the cross_group_correlations to only rows where Variable_1 is the current target
    subset_corr = cross_group_correlations[
        (cross_group_correlations['Variable_1'] == target) | (cross_group_correlations['Variable_2'] == target)
        ].copy()

    # Keeping only relevant columns and renaming 'Variable_2' to 'Feature' so we can merge
    subset_corr = subset_corr[['Variable_2', 'Correlation', 'Interaction']].rename(columns={'Variable_2': 'Feature'})

    # Merging this info into the significant_df
    summary_df = summary_df.merge(subset_corr, on='Feature', how='left')
    
    # Optionally filtering for statistical significance (p < 0.05)
    # This part is commented out since we end up with few enough features that saving them all is a non-issue,
    # and we can filter by these parameters later on anyway.
    # significant_df = summary_df[summary_df['P_Value'] < 0.05].sort_values(by='LASSO_Coef')
    
    significance_results[target] = summary_df.sort_values(by='P_Value')


Analyzing Significance for: Age-adjusted ER Visit Rate for Asthma per 10,000 People
Counties available for this model: 1512
LASSO selected 85 features. Running OLS for significance...
Adjusted R^2: 0.6117

Analyzing Significance for: Age-adjusted Hospitalization Rate for Asthma per 10,000 People
Counties available for this model: 1096
LASSO selected 33 features. Running OLS for significance...
Adjusted R^2: 0.4524


In [20]:
# Printing out the tables for markdown
# These tables are available in the readme page of the GitHub project, and are filtered
# to include only p-values less than 0.05 (statistically significant) 
# and VIF less than 10 (potentially meaningful on its own, not too collinear with many other features)
for i in asthma_columns:
    target = (significance_results[i][
        (significance_results[i]['Correlation'].notnull()) 
        & (significance_results[i]['P_Value'] < 0.05)
        & (significance_results[i]['VIF'] < 10)
        ])

    # Sorting by absolute value of OLS. When printing we will still print the real value of the OLS,
    # but a very negative OLS value is more significant than a very low positive value as a negative driver,
    # so this step, while trivial, is crucial for healthy observation
    sorted_df = target.sort_values(
        by='OLS_Coef', 
        key=lambda x: x.abs(), 
        ascending=False
    )[['Feature', 'LASSO_Coef', 'OLS_Coef', 'P_Value', 'VIF', 'Correlation']]

    print('='*80)
    print(i)
    print(sorted_df.head(100).to_markdown(index=False))
    print('='*80)
    print('')

Age-adjusted ER Visit Rate for Asthma per 10,000 People
| Feature                                                                                                                                       |   LASSO_Coef |   OLS_Coef |     P_Value |     VIF |   Correlation |
|:----------------------------------------------------------------------------------------------------------------------------------------------|-------------:|-----------:|------------:|--------:|--------------:|
| Demographics => % African American                                                                                                            |     5.90663  |   6.45007  | 2.6466e-10  | 9.88573 |    0.535384   |
| (Household Composition/Disability) Percentage of persons aged 17 and younger estimate MOE, 2014-2018 ACS 2018 DESCRIPTION [MP_AGE17]          |    -2.45136  |  -3.33644  | 8.09441e-10 | 2.80024 |   -0.189766   |
| Median household income => Household Income                                           

# Saving the Result

In [24]:
corr_table_folder = r'C:\Users\mpola\asthma analysis\files\\correlation tables'

# Saving the full analysis tables
for column in asthma_columns:
    results = significance_results[column].sort_values(
        by='OLS_Coef', 
        key=lambda x: x.abs(), 
        ascending=False
    )

    results.to_csv(f'{corr_table_folder}\Results ({column}).csv')

# Saving a singular .xlsx file containing all the correlation tables as worksheets using XlsxWriter
import xlsxwriter

workbook = xlsxwriter.Workbook(f'{corr_table_folder}\correlation_table.xlsx')

for i in cross_group_correlations.sort_values('Interaction')['Interaction'].unique():
    sheet = pl.from_pandas(cross_group_correlations[cross_group_correlations['Interaction']==i]).select(
        ['Correlation',
        'Variable_1',
        'Variable_2']
    )
    sheet.write_excel(workbook=workbook, worksheet=i)

workbook.close()

